In [1]:
import pandas as pd 
import numpy as np 
import pickle 
from sklearn.model_selection import train_test_split

In [14]:
# Logical - easy
with open(f'data/datasets/curated_dataset_full.pkl', 'rb') as file:
    curated_dataset = pickle.load(file)
with open(f'data/datasets/neg_dataset.pkl', 'rb') as file:
    negated = pickle.load(file)
with open(f'data/datasets/disj_dataset.pkl', 'rb') as file:
    disjunction = pickle.load(file)
with open(f'data/datasets/conj_dataset.pkl', 'rb') as file:
    conjunction = pickle.load(file)
# Logical - inference
with open(f'data/datasets/larger_than_inference.pkl', 'rb') as file:
    lt_inference = pickle.load(file)
with open(f'data/datasets/smaller_than_inference.pkl', 'rb') as file:
    st_inference = pickle.load(file)
with open(f'data/datasets/cities_inference.pkl', 'rb') as file:
    cities_inference = pickle.load(file)
with open(f'data/datasets/companies_inference.pkl', 'rb') as file:
    companies_inference = pickle.load(file)
with open(f'data/datasets/commonclaim_inference.pkl', 'rb') as file:
    cc_inference = pickle.load(file)
with open(f'data/datasets/counterfact_inference.pkl', 'rb') as file:
    cf_inference = pickle.load(file)
# Other datasets
with open(f'data/datasets/entailment_new.pkl', 'rb') as file:
    entailment_new = pickle.load(file)

In [56]:
def get_data_split(task: str, curated_dataset, probe_config, other_dataset=None, cutoff=1500):


    if task in ['negation', 'disjunction', 'conjunction']:
        # In this case curated dataset is the logical dataset + the remainder
        remainder = curated_dataset[~curated_dataset['filename'].isin(['common_claim_true_false.csv', 'companies_true_false.csv', 'counterfact_true_false.csv'])]
        curated_dataset = pd.concat([remainder, other_dataset])
        train_set, test_set = train_test_split(curated_dataset, test_size=0.2, random_state=42, stratify=curated_dataset['filename'])
        test_df = test_set.dropna()
        
    elif task == 'inference':
        remainder = curated_dataset[~curated_dataset['filename'].isin(['common_claim_true_false.csv', 'companies_true_false.csv', 'counterfact_true_false.csv',
                                                                      'cities.csv', 'cities_cities_conj.csv', 'cities_cities_disj.csv', 'neg_cities.csv'
                                                                      ])]
        curated_dataset = pd.concat([remainder, other_dataset])
        train_set, test_set = train_test_split(curated_dataset, test_size=0.2, random_state=42, stratify=curated_dataset['filename'])
        test_df = test_set.dropna()
        

    # Trim for batch size
    train_set = train_set.iloc[:-(len(train_set) % probe_config), :]
    X_clean_train = list(train_set['statement'])
    y_clean_train = list(train_set['label'])
    
    return (X_clean_train, y_clean_train, test_df)